In [ ]:
import pandas as pd
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

# Survey data processing

In [64]:
data = pd.read_csv('../data/original/survey/survey.csv', parse_dates=['StartDate'], usecols = ['StartDate','Q2','Q3','Q4','Q7','Q8','Q9','Q10','Q11.a','Q11.b','Q12'])

convert survey data to correct timezone

In [65]:
data['StartDate'] = data['StartDate'].dt.tz_localize('America/Denver')
data['StartDate'] = data['StartDate'].dt.tz_convert('Europe/Amsterdam')

convert location question data to known place ids

In [66]:
def assign_place_id(row):
    if row['Q7'] == 'Ground floor' and row['Q8'] == 'Round tables by the three plants (across wooden staircase)':
        return 1
    elif row['Q7'] == 'Ground floor' and row['Q8'] == 'Study corner next to the plant wall':
        return 2
    elif row['Q7'] == '1st Floor' and row['Q9'] == 'Tables on the landing (with wooden floor) accessible by the black staircase':
        return 3
    elif row['Q7'] == '1st Floor' and row['Q9'] == 'Yellow/white chairs & tables (besides the wooden staircase)':
        return 4
    else:
        return 0

data['place_id'] = data.apply(assign_place_id, axis=1)

drop unneeded columns and rename the rest appropriately

In [67]:
data = data.drop(columns=['Q7','Q8','Q9','Q10','Q11.a'])
data = data.rename(columns={'Q2':'emotion','Q3':'comfort','Q4':'frequent_use','Q11.b':'student','Q12':'activity'})

Properly assign numeric values:

1. Learning / Working by yourself
2. Learning / Working in a group
3. Taking a break

In [68]:
def assign_activity_id(row):
    if row['activity'] == 'Learning / Working by yourself':
        return 1
    elif row['activity'] == 'Learning / Working in a group':
        return 2
    elif row['activity'] == 'Taking a break':
        return 3
    else:
        return 0

data['activity'] = data.apply(assign_activity_id, axis=1)


data['frequent_use'] = data['frequent_use'].map(dict(Yes=1, No=0))
data['student'] = data['student'].map(dict(Yes=1, No=0))

convert survey strings to lowercase for better processsing

In [69]:
data['emotion'] = data['emotion'].astype(str)
data['emotion'] = data['emotion'].str.lower()
data['comfort'] = data['comfort'].astype(str)
data['comfort'] = data['comfort'].str.lower()

# NLP

Tokenize words and stencences and stem words for further analysis

In [71]:
def stem_words(words):
    stemmer = nltk.stem.PorterStemmer()
    return [stemmer.stem(word) for word in words]

data['emotion_tokens'] = data['emotion'].apply(nltk.word_tokenize)
data['comfort_tokens'] = data['comfort'].apply(nltk.word_tokenize)

data['emotion_tokens_sent'] = data['emotion'].apply(nltk.sent_tokenize)
data['comfort_tokens_sent'] = data['comfort'].apply(nltk.sent_tokenize)

data['emotion_tokens_stem'] = data['emotion_tokens'].apply(stem_words)
data['comfort_tokens_stem'] = data['comfort_tokens'].apply(stem_words)


In [72]:
data

,StartDate,emotion,comfort,activity,student,frequent_use,place_id,emotion_tokens,comfort_tokens,emotion_tokens_sent,comfort_tokens_sent,emotion_tokens_stem,comfort_tokens_stem
0,2023-05-02 16:21:52+02:00,i feel pretty calm and relaxed.,i don’t notice any uncomfortable factors. temp...,1,1.0,0.0,0,"[i, feel, pretty, calm, and, relaxed, .]","[i, don, ’, t, notice, any, uncomfortable, fac...",[i feel pretty calm and relaxed.],"[i don’t notice any uncomfortable factors., te...","[i, feel, pretti, calm, and, relax, .]","[i, don, ’, t, notic, ani, uncomfort, factor, ..."
1,2023-05-03 13:12:22+02:00,i really like it! it’s fairly quiet right now ...,very comfortable! perfect temperature,2,1.0,1.0,3,"[i, really, like, it, !, it, ’, s, fairly, qui...","[very, comfortable, !, perfect, temperature]","[i really like it!, it’s fairly quiet right no...","[very comfortable!, perfect temperature]","[i, realli, like, it, !, it, ’, s, fairli, qui...","[veri, comfort, !, perfect, temperatur]"
2,2023-05-03 13:22:43+02:00,quiet,8/10,2,0.0,0.0,1,[quiet],[8/10],[quiet],[8/10],[quiet],[8/10]
3,2023-05-03 13:12:38+02:00,"social, but still a nice place to study. it ca...",it get cold,2,1.0,0.0,0,"[social, ,, but, still, a, nice, place, to, st...","[it, get, cold]","[social, but still a nice place to study., it ...",[it get cold],"[social, ,, but, still, a, nice, place, to, st...","[it, get, cold]"
4,2023-05-04 12:49:19+02:00,"having computer issues so stressed, not really...",pretty good but it's a bit too loud and open t...,1,1.0,0.0,2,"[having, computer, issues, so, stressed, ,, no...","[pretty, good, but, it, 's, a, bit, too, loud,...","[having computer issues so stressed, not reall...",[pretty good but it's a bit too loud and open ...,"[have, comput, issu, so, stress, ,, not, reall...","[pretti, good, but, it, 's, a, bit, too, loud,..."
5,2023-05-04 12:48:52+02:00,"it's very quiet here right now, so i am feelin...","it's a little chilly, but i don't particularly...",1,1.0,0.0,3,"[it, 's, very, quiet, here, right, now, ,, so,...","[it, 's, a, little, chilly, ,, but, i, do, n't...","[it's very quiet here right now, so i am feeli...","[it's a little chilly, but i don't particularl...","[it, 's, veri, quiet, here, right, now, ,, so,...","[it, 's, a, littl, chilli, ,, but, i, do, n't,..."
6,2023-05-04 13:34:45+02:00,"pretty calm and no emotions, just finished my ...",7/10,3,1.0,1.0,2,"[pretty, calm, and, no, emotions, ,, just, fin...",[7/10],"[pretty calm and no emotions, just finished my...",[7/10],"[pretti, calm, and, no, emot, ,, just, finish,...",[7/10]
7,2023-05-08 08:48:45+02:00,"i'm feeling alright at the moment, only a bit ...",pretty comfortable,3,1.0,1.0,1,"[i, 'm, feeling, alright, at, the, moment, ,, ...","[pretty, comfortable]","[i'm feeling alright at the moment, only a bit...",[pretty comfortable],"[i, 'm, feel, alright, at, the, moment, ,, onl...","[pretti, comfort]"
8,2023-05-08 09:59:32+02:00,"it’s quite relaxing, although the students hav...",although this answer feels somewhat lazily con...,1,1.0,1.0,3,"[it, ’, s, quite, relaxing, ,, although, the, ...","[although, this, answer, feels, somewhat, lazi...","[it’s quite relaxing, although the students ha...",[although this answer feels somewhat lazily co...,"[it, ’, s, quit, relax, ,, although, the, stud...","[although, thi, answer, feel, somewhat, lazili..."
9,2023-05-08 10:23:15+02:00,the openness of the space and the plants on th...,"the chairs feel a little hard, but not too har...",1,1.0,1.0,4,"[the, openness, of, the, space, and, the, plan...","[the, chairs, feel, a, little, hard, ,, but, n...",[the openness of the space and the plants on t...,"[the chairs feel a little hard, but not too ha...","[the, open, of, the, space, and, the, plant, o...","[the, chair, feel, a, littl, hard, ,, but, not..."
